## MiniProjeto - Analisando Dados do Uber com Spark

##### Versões utilizadas: Java JDK 11 e Apache Spark 2.4.2

Dataset: https://github.com/fivethirtyeight/uber-tlc-foil-response

Esse conjunto de dados contém dados de mais de 4,5 milhões de captações Uber na cidade de Nova York de abril a setembro de 2014 e 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015.

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passaram pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

In [30]:
from pandas import read_csv

In [31]:
df_uber = read_csv('data/uber.csv')

In [32]:
type(df_uber)

pandas.core.frame.DataFrame

In [33]:
df_uber.head(2)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765


In [34]:
# criando a rdd
rdd_uber = sc.textFile("data/uber.csv")

In [35]:
type(rdd_uber)

pyspark.rdd.RDD

In [36]:
rdd_uber.count()

355

In [37]:
rdd_uber.first()

'dispatching_base_number,date,active_vehicles,trips'

- 1) Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

In [38]:
rdd_uber_linhas = rdd_uber.map(lambda line: line.split(','))

In [39]:
rdd_uber_linhas.first()

['dispatching_base_number', 'date', 'active_vehicles', 'trips']

In [40]:
# para as quantidades de bases de carros
rdd_uber_linhas.map(lambda line: line[0]).distinct().count() -1

6

In [41]:
# quais são as bases
rdd_uber_linhas.map(lambda line: line[0]).distinct().collect() 

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

- 2) Qual o total de veículos que passaram pela base B02617?

In [42]:
rdd_uber_B02617 = rdd_uber_linhas.filter(lambda line: "B02617" in line)

In [43]:
rdd_uber_B02617.count()

59

- 3) Qual o total de corridas por base? Apresente de forma decrescente.

In [47]:
uberRDD2 = sc.textFile("data/uber.csv").filter(lambda line: "base" not in line).map(lambda line:line.split(","))

In [48]:
uberRDD2.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [49]:
# Aplicando redução para calcular o total por base, em ordem decrescente
uberRDD2.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).takeOrdered(10, key = lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]